读取 LogRecord

[read-LogRecord](./docs/images/read-logrecord.png)

上图信息是存储在数据文件中的一条 LogRecord 的结构，我们可以将其分成两部分：
- 一是头部信息，存储了一些元数据，例如 crc 校验值、Type 类型、Key 的大小、Value 的大小
- 二是包含用户实际的 key、value 部分

Header 部分

crc 占 4 个字节
Type 占 1 个字节
Key size 变长
Value size 变长

> key size 和 value size 变长的原因是为了节省空间，如果 key size 是 u32 类型的话，如果不使用变长，将固定占据4个字节，但是有的时候我们 key 的长度很小，例如长度为 5，那么只需要一个字节就够了

读取的时候，需要判断读取的偏移 + LogRecord 的最大头部字节数，是不是超过了文件大小，针对这个 case 需要特殊处理。